In [1]:
import os
os.chdir('../')
# esse comando é melhor do que sys.path.insert(0, '../') pois permite interagir com arquivos do diretorio acima

In [2]:
import pickle
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 167) # opcional, questão de visualização para debugar caso 1

In [3]:
from pynnystock.FileManager import FileManager
from pynnystock.Ativo import Ativo
from pynnystock.Parameters import Parameters
from pynnystock.StatsGatherer import StatsGatherer
from pynnystock.StratsMaestro import StratsMaestro
from pynnystock.Simulator import Simulator
from pynnystock.BootstrapSimulator import BootstrapSimulator
from pynnystock.OptimizerSimulator import OptimizerSimulator

In [4]:
fm = FileManager()

In [5]:
pars = Parameters()

In [6]:
sg = StatsGatherer(pars)

##### Na primeira vez precisa gerar o pickle de ativo_dia_list.pkl
##### Somente executar esse código se o pickle ativo_dia_list.pkl não tiver sido gerado

In [7]:
# abre arquivo com freefloats
names_pennystocks = fm.getNames()
names_free_float = fm.getFreeFloatNames()
names = list(set(names_pennystocks) & set(names_free_float))
names.sort()

In [ ]:
# gera ativo_dia_list (adl), que é uma lista com resumo dos dias que vai nos auxiliar a filtrar os dias de interesse
# também vai auxiliar na análise de ativos-dia
adl = []
for n in names[0:10]:
#for n in names:
    print(n)
    try:
        a = Ativo(n, fm[n], sg)
        for intraday in a.intraDays:
            d = {'name': a.name,
                 'date': intraday.date,
                 'freefloat': fm.getFreeFloat(n),
                 'stats': intraday.stats}
            adl.append( d )
    except IndexError:
        print("Ativo sem nenhum dado ou dado inconsistente")

In [ ]:
with open('AtivoDiaList.pkl', 'wb') as filehandle: # w de write e b de binary
    pickle.dump(adl,filehandle)

##### Com o pickle ativo_dia_list.pkl gerado, prosseguir daqui

In [ ]:
with open('AtivoDiaList.pkl', 'rb') as filehandle:
    ativo_dia_list = pickle.load(filehandle)

In [ ]:
sm = StratsMaestro(pars)

In [ ]:
sim = Simulator(fm,ativo_dia_list,pars,sm,sg)

In [ ]:
bss = BootstrapSimulator(sg)

In [ ]:
os = OptimizerSimulator(sim, bss, sg)

In [ ]:
# pars.setFilterParameters(gap_threshold=0.1, F_high_threshold=9)
pars.setFilterParameters(prevol_threshold=15000000)
sim.runFiltering()

In [ ]:
# pars.setSimParameters(start_money = 10000,allocation=0.1)
# pars.setAlgoParameters(short_after=0, exit_target=0.4, exit_stop=0.4)
# pars.setAlgoParameters(short_after=0.2)

sim.runSimulation()

#sim.saveTrades('case1_trades.pkl')
# sim.openTrades('case1_trades.pkl')

In [ ]:
dfd = sg.filtereddf # DataFrame of filtered ativo-dias
dft = sg.tradesdf # DataFrame of trades
dfes = sg.extrastatsdf # DataFrame of extra statistics

In [ ]:
dfd

##### Algumas análises da simulação

In [ ]:
sg.printSimResults()

In [ ]:
sg.plotHistMinsToTrade()

In [ ]:
sg.plotEquityCurve()

##### Análise de algumas estratificações

In [ ]:
dftr = pd.merge(dft,dfd,how='inner',on=['name','date'])
dftr = dftr[['name','date','gap','factorF','profit','cum_profit']]
dftr

In [ ]:
bins = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2]
gap = np.clip( dftr['gap'], bins[0], bins[-1] )
dftr.groupby(pd.cut(gap, bins)).agg(
    winrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0)/(sum(x>=0) + sum(x<0)) ),
    lossrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0)/(sum(x>=0) + sum(x<0)) ),
    mean=pd.NamedAgg(column='profit',aggfunc=lambda x: x.mean()),
    mean_of_pos=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x>=0].mean()),
    mean_of_neg=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x<0].mean()),
    count=pd.NamedAgg(column='profit',aggfunc='count'),
    positives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0) ),
    negatives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0) )
)

In [ ]:
dftr.groupby(dftr['date'].dt.to_period('M')).agg(
    winrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0)/(sum(x>=0) + sum(x<0)) ),
    lossrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0)/(sum(x>=0) + sum(x<0)) ),
    mean=pd.NamedAgg(column='profit',aggfunc=lambda x: x.mean()),
    mean_of_pos=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x>=0].mean()),
    mean_of_neg=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x<0].mean()),
    count=pd.NamedAgg(column='profit',aggfunc='count'),
    positives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0) ),
    negatives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0) )
)

##### Analisando o Dataframe com extra statistics

In [ ]:
# aqui podemos fazer alguma análise, basta fazer inner join de dfes com os outros dataframes dfd, dft
dfes

##### Rodando Bootstrap

In [ ]:
bss.runBootstrap()

In [ ]:
sg.printBootstrapResults()

##### Códigos para rodar combinações de parâmetros e otimizar.

In [ ]:
# oficial que gostaríamos de explorar
os.runSimulationGroup(
    prevol_threshold = [800000],
    exit_target=[0.1, 0.2, 0.3, 0.4], 
    exit_stop=[0.1, 0.2, 0.3, 0.4], 
    short_after=[0, 0.1, 0.2, 0.3],
    gap_threshold=[0, 0.1, 0.2],
    F_high_threshold = [1,2,3,4,5]
)

In [ ]:
# rodar para debugar
os.runSimulationGroup(
    prevol_threshold = [15000000],
    exit_target=[0.2, 0.3], 
    exit_stop=[0.3], 
    short_after=[0, 0.1],
    gap_threshold=[0.1],
    F_high_threshold = [1]
)

In [ ]:
# sg.saveGroupResults('teste1.pkl')
sg.appendGroupResults('teste1.pkl')
# sg.openGroupResults('teste1.pkl')

In [ ]:
sg.groupResults

##### Descrição simulações já rodadas

##### Testes temporários em geral. Desconsiderar essa parte do notebook.

In [ ]:
sim.fad[0]['stats']['highCoreTime']

In [ ]:
import datetime

In [ ]:
t2 = sim.fad[0]['stats']['highCoreTime']

In [ ]:
t1 = datetime.time(9,31)

In [ ]:
datetime.datetime.combine(date.today(),t1)

In [ ]:
datetime.combine(date.today(),t2)

In [ ]:
sim.fad

In [ ]:
datetime.date(2020, 7, 24)

In [ ]:
sg

In [ ]:
intra = Ativo.initIntradayFromDate('ABUS',fm['ABUS'],datetime.date(2020, 7, 24), sg)

In [ ]:
intra._core[0]

In [ ]:
intra._core[0]['time'].time()

In [ ]:
# como vamos calcular average price?
# vi em algum lugar sugestão de (open+high+close)/3
bar = intra._core[2]

In [ ]:
(bar['high']+bar['low']+bar['close'])/3

In [ ]:
(bar['high']+bar['low']+bar['close'])/3

In [ ]:
intra

In [ ]:
from pynnystock.indicators.VWAP import VWAP

In [ ]:
vwap = VWAP(intra)

In [ ]:
print(vwap)

In [ ]:
vwap.PQt

In [ ]:
for b in intra._core:
    vwap.getNextVWAP(b)
    print(vwap)